<a href="https://colab.research.google.com/github/LC1332/personality-text-generation/blob/main/lulu_exp/%E6%A0%B9%E6%8D%AE%E9%97%AE%E9%A2%98%EF%BC%8C%E8%8E%B7%E5%8F%96chatbot%E7%9A%84response%E5%9B%9E%E5%A4%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

implemented by 李鲁鲁

- [x] 读取问题
- [x] 载入chatbot， temp = 0
- [x] 依次作答，保存到drive

In [ ]:
input_name = 'questions_verified.jsonl'

import json

datas = []

with open(input_name, encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        datas.append(data)

In [ ]:
print(len(datas))

106


让我们来载入Chatbot

In [ ]:
import os

key = "sk-OSq"
key_bytes = key.encode()
os.environ["OPENAI_API_KEY"] = key_bytes.decode('utf-8')

In [ ]:
!pip -q install gradio wget openai chromadb==0.4.7 langchain transformers tiktoken zhipuai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 79.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.5/415.5 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 118.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev

/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 731, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 731 (delta 19), reused 18 (delta 8), pack-reused 693
Receiving objects: 100% (731/731), 105.69 MiB | 16.17 MiB/s, done.
Resolving deltas: 100% (349/349), done.
/content/Haruhi-2-Dev


In [ ]:
import zipfile
from ChatHaruhi import ChatHaruhi

In [ ]:
NAME_DICT = {'汤师爷': 'tangshiye', '慕容复': 'murongfu', '李云龙': 'liyunlong', 'Luna': 'Luna', '王多鱼': 'wangduoyu',
             'Ron': 'Ron', '鸠摩智': 'jiumozhi', 'Snape': 'Snape',
             '凉宫春日': 'haruhi', 'Malfoy': 'Malfoy', '虚竹': 'xuzhu', '萧峰': 'xiaofeng', '段誉': 'duanyu',
             'Hermione': 'Hermione', 'Dumbledore': 'Dumbledore', '王语嫣': 'wangyuyan',
             'Harry': 'Harry', 'McGonagall': 'McGonagall', '白展堂': 'baizhantang', '佟湘玉': 'tongxiangyu',
             '郭芙蓉': 'guofurong', '旅行者': 'wanderer', '钟离': 'zhongli',
             '胡桃': 'hutao', 'Sheldon': 'Sheldon', 'Raj': 'Raj', 'Penny': 'Penny', '韦小宝': 'weixiaobao',
             '乔峰': 'qiaofeng', '神里绫华': 'ayaka', '雷电将军': 'raidenShogun', '于谦': 'yuqian'}


ai_roles_obj = {}

for ai_role_en in NAME_DICT.values():
    zip_file_path = f"/content/Haruhi-2-Dev/data/character_in_zip/{ai_role_en}.zip"
    if not os.path.exists(zip_file_path):
        # os.remove(zip_file_path)
        print('unfound zip file ', zip_file_path)
        continue

    destination_folder = f"characters/{ai_role_en}"

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(destination_folder)

    db_folder = f"./characters/{ai_role_en}/content/{ai_role_en}"
    system_prompt = f"./characters/{ai_role_en}/content/system_prompt.txt"
    ai_roles_obj[ai_role_en] = ChatHaruhi(system_prompt=system_prompt,
                        llm="openai",
                        story_db=db_folder,
                        verbose=True)
    # break

print(ai_roles_obj)

{'tangshiye': <ChatHaruhi.ChatHaruhi.ChatHaruhi object at 0x781b0759f940>, 'murongfu': <ChatHaruhi.ChatHaruhi.ChatHaruhi object at 0x781b0759f9a0>, 'liyunlong': <ChatHaruhi.ChatHaruhi.ChatHaruhi object at 0x781aed9581f0>, 'Luna': <ChatHaruhi.ChatHaruhi.ChatHaruhi object at 0x781aee5c4910>, 'wangduoyu': <ChatHaruhi.ChatHaruhi.ChatHaruhi object at 0x781aee5c47c0>, 'Ron': <ChatHaruhi.ChatHaruhi.ChatHaruhi object at 0x781aee601b10>, 'jiumozhi': <ChatHaruhi.ChatHaruhi.ChatHaruhi object at 0x781aed911300>, 'Snape': <ChatHaruhi.ChatHaruhi.ChatHaruhi object at 0x781b0759f970>, 'haruhi': <ChatHaruhi.ChatHaruhi.ChatHaruhi object at 0x781aed9117e0>, 'Malfoy': <ChatHaruhi.ChatHaruhi.ChatHaruhi object at 0x781aed8d7ac0>, 'xuzhu': <ChatHaruhi.ChatHaruhi.ChatHaruhi object at 0x781aed9126e0>, 'xiaofeng': <ChatHaruhi.ChatHaruhi.ChatHaruhi object at 0x781aed912650>, 'duanyu': <ChatHaruhi.ChatHaruhi.ChatHaruhi object at 0x781aed912b60>, 'Hermione': <ChatHaruhi.ChatHaruhi.ChatHaruhi object at 0x781aed9130

把所有bot的温度调为0

In [ ]:
for key in ai_roles_obj.keys():
    ai_roles_obj[key].llm.chat.temperature = 0

In [ ]:
tester = {'汤师爷': '张牧之', '慕容复': '王语嫣', \
          '李云龙': '赵刚', 'Luna': 'Harry', '王多鱼': '夏竹',
          'Ron': 'Hermione', '鸠摩智': '慕容复', 'Snape': 'Dumbledore',
             '凉宫春日': '阿虚', 'Malfoy': 'Crabbe', \
          '虚竹': '乔峰', '萧峰': '阿朱', '段誉': '乔峰',\
             'Hermione': 'Harry', 'Dumbledore': 'McGonagall', '王语嫣': '段誉',\
             'Harry': 'Hermione', 'McGonagall': 'Dumbledore', '白展堂': '佟湘玉',\
           '佟湘玉': '白展堂',
             '郭芙蓉': '白展堂', '旅行者': '派蒙', '钟离': '旅行者',
             '胡桃': '旅行者', 'Sheldon': 'Leonard', 'Raj': 'Leonard', 'Penny': 'Leonard', \
          '韦小宝': '双儿',
             '乔峰': '阿朱', '神里绫华': '旅行者', '雷电将军': '旅行者', '于谦': '郭德纲'}


In [ ]:
target = '凉宫春日'

en_name = NAME_DICT[target]

chatbot = ai_roles_obj[en_name]

tester_name = tester[target]

question = datas[0]['question']

response = chatbot.chat(role = tester_name, text = question)

正在下载Luotuo-Bert


A new version of the following files was downloaded from https://huggingface.co/silk-road/luotuo-bert-medium:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Luotuo-Bert下载完毕


In [ ]:
print(question)
print(response)

你有时会感到寂寞或忧郁吗？
春日:「寂寞？忧郁？我可是凉宫春日，怎么可能会有这种感觉呢！我总是充满活力和热情的，我有SOS团的事情要忙，还有各种奇妙的事件等着我们解决呢！寂寞和忧郁对我来说只是陌生的词汇而已！」


In [ ]:
query_list = []

for target in NAME_DICT.keys():
    for question_id in range(len(datas)):
        query_list.append((target, question_id))

print(len(query_list))


3392


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

save_name = '/content/drive/MyDrive/Psy/psy_test_gpt.txt'


if_first = True

In [ ]:
print(query_list[:5])

[('汤师爷', 0), ('汤师爷', 1), ('汤师爷', 2), ('汤师爷', 3), ('汤师爷', 4)]


来加个断点续传吧

In [ ]:

import json

role_id = {}

previous_response = []

with open(save_name, encoding='utf-8') as f:
    for line in f:
        data = json.loads(line)
        # id = data['id']

        previous_response.append(data)


In [26]:
from tqdm import tqdm

count = 0


for target, question_id in tqdm(query_list):

    if count < len(previous_response):
        count = count + 1
        continue


    en_name = NAME_DICT[target]

    chatbot = ai_roles_obj[en_name]

    chatbot.dialogue_history = []

    data = datas[question_id]

    question = data['question']

    tester_name = tester[target]

    response = chatbot.chat(role = tester_name, text = question)

    file_mode = 'w' if if_first else 'a'

    if_first = False

    with open(save_name, file_mode, encoding= 'utf-8') as f:
        data = {
            'id':data['id'],
            'question':data['question'],
            'response':response,
            'factor':data['factor'],
            'sub_factor':data['sub_factor'],
            'test_role':tester_name
        }
        json.dump(data, f, ensure_ascii=False)
        f.write('\n')


100%|██████████| 3392/3392 [7:02:43<00:00,  7.48s/it]


In [27]:
print(datas[question_id])

{'id': 109, 'question': '我开始工作时也常需要先进入状态。你是如何快速投入工作的呢?有什么诀窍可以帮助自己更高效地开始工作吗?', 'factor': 'conscientiousness', 'sub_factor': 'C6 (Deliberation)', 'if_pos': 'synthesis', 'if_plausible': True, 'reason': '如果被试回答他们有一些方法和技巧可以帮助自己快速进入工作状态，并提到了一些策略来提高工作效率，那就意味着他们在C6（审慎性）方面得分较高。这表明他们在行动之前会仔细考虑，并具有较高的决策思考能力。', 'prob_repeat': '我开始工作时也常需要先进入状态。你是如何快速投入工作的呢?有什么诀窍可以帮助自己更高效地开始工作吗?'}


In [ ]:

    # if question_id > 3:
    #     break

    # print(question)
    # print(response)

    # break
